# SweetEscape – Notebook 00: Preprocessing & Feature Engineering

In diesem Notebook wird der Rohdatensatz geladen und es werden wenige, medizinisch motivierte Features abgeleitet.
Anschließend speichern wir den verarbeiteten Datensatz unter `data/processed/diabetes_fe.csv`.

**Warum?**
Damit das Modelltraining (Notebook 01) reproduzierbar ist und nicht von Notebook-internen Zwischenschritten abhängt.

## Setup & Projektpfade

Wir definieren zentrale Pfade und Konstanten, damit das Notebook unabhängig vom Ausführungsort
reproduzierbar bleibt und keine hartkodierten Dateipfade enthält.

In [1]:
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0]  # notebooks/ → Projekt-Root
RAW_PATH = PROJECT_ROOT / "data" / "raw" / "diabetes_raw.csv"
OUT_PATH = PROJECT_ROOT / "data" / "processed" / "diabetes_fe.csv"

TARGET = "Diabetes_012"

RAW_PATH, OUT_PATH

(WindowsPath('C:/Users/Jan/DataspellProjects/SweetEscape/data/raw/diabetes_raw.csv'),
 WindowsPath('C:/Users/Jan/DataspellProjects/SweetEscape/data/processed/diabetes_fe.csv'))

## Laden der Rohdaten

Zunächst laden wir den originalen Datensatz aus `data/raw`.
Die Rohdaten bleiben unverändert und dienen als Referenz.

In [2]:
df = pd.read_csv(RAW_PATH)

print("Shape:", df.shape)
df.head()

Shape: (253680, 22)


,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


## Erste Validierung der Daten

Wir überprüfen:
- ob die Zielvariable existiert
- die Klassenverteilung
- grobe Datentypen

Dies ist wichtig, um spätere Modellmetriken korrekt interpretieren zu können.

In [3]:
assert TARGET in df.columns, "Zielvariable fehlt!"

display(df[TARGET].value_counts().sort_index())
df.dtypes.head(10)

Diabetes_012
0.0    213703
1.0      4631
2.0     35346
Name: count, dtype: int64

Diabetes_012            float64
HighBP                  float64
HighChol                float64
CholCheck               float64
BMI                     float64
Smoker                  float64
Stroke                  float64
HeartDiseaseorAttack    float64
PhysActivity            float64
Fruits                  float64
dtype: object

## Feature-Engineering-Strategie

Der Datensatz ist bereits stark vorverarbeitet.
Daher führen wir bewusst nur **moderates Feature Engineering** durch:

- Aggregation medizinisch zusammenhängender Risiken
- Ableitung einfacher binärer Risikosignale
- Keine Veränderung der Zielvariable
- Kein Oversampling / Scaling (gehört ins Training)

Ziel ist Interpretierbarkeit statt maximaler Modellkomplexität.

## Implementierung des Feature Engineerings

Wir kapseln alle Feature-Engineering-Schritte in einer Funktion,
damit sie reproduzierbar und klar nachvollziehbar bleiben.

In [4]:
def feature_engineering(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    df[TARGET] = df[TARGET].astype(int)

    # Inaktivität
    df["inactive"] = (1 - df["PhysActivity"]).astype(int)

    # Herz-Kreislauf-Risiko
    df["cardio_risk_sum"] = df[
        ["HighBP", "HighChol", "HeartDiseaseorAttack", "Stroke"]
    ].sum(axis=1)

    # Lifestyle-Risiko
    df["low_fruits"] = (1 - df["Fruits"]).astype(int)
    df["low_veggies"] = (1 - df["Veggies"]).astype(int)

    df["lifestyle_risk_sum"] = df[
        ["Smoker", "HvyAlcoholConsump", "inactive", "low_fruits", "low_veggies"]
    ].sum(axis=1)

    # Schlechter Gesundheitszustand
    df["poor_health"] = (df["GenHlth"] >= 4).astype(int)

    # Gesamtbelastung aus schlechten Tagen
    df["mental_physical_burden"] = df["MentHlth"] + df["PhysHlth"]

    return df

## Anwenden des Feature Engineerings

Wir wenden das Feature Engineering an und prüfen,
welche neuen Features hinzugekommen sind.

In [5]:
df_fe = feature_engineering(df)

print("Vorher:", df.shape)
print("Nachher:", df_fe.shape)

[c for c in df_fe.columns if c not in df.columns]

Vorher: (253680, 22)
Nachher: (253680, 29)


['inactive',
 'cardio_risk_sum',
 'low_fruits',
 'low_veggies',
 'lifestyle_risk_sum',
 'poor_health',
 'mental_physical_burden']

## Qualitätskontrollen

Vor dem Speichern prüfen wir:
- fehlende Werte
- ob Binärfeatures wirklich 0/1 sind

So vermeiden wir stille Fehler im Modelltraining.

In [6]:
df_fe.isna().sum().loc[lambda x: x > 0]

Series([], dtype: int64)

## Speichern der verarbeiteten Daten

Der finale, feature-engineerte Datensatz wird nun gespeichert.
Dieser Datensatz ist die einzige Grundlage für das Modelltraining.

In [7]:
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
df_fe.to_csv(OUT_PATH, index=False)

print("Gespeichert:", OUT_PATH)

Gespeichert: C:\Users\Jan\DataspellProjects\SweetEscape\data\processed\diabetes_fe.csv


In [8]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0]  # notebooks/ -> Projekt-Root
TARGET = "Diabetes_012"

# Falls du df_fe so genannt hast (wie in unserem Notebook 00):
# df_fe enthält den finalen Feature-Engineering Output
FEATURES_OUT = PROJECT_ROOT / "data" / "processed" / "features_used.txt"

feature_cols = [c for c in df_fe.columns if c != TARGET]

FEATURES_OUT.parent.mkdir(parents=True, exist_ok=True)
FEATURES_OUT.write_text("\n".join(feature_cols), encoding="utf-8")

print("✅ Gespeichert:", FEATURES_OUT)
print("Anzahl Features:", len(feature_cols))

✅ Gespeichert: C:\Users\Jan\DataspellProjects\SweetEscape\data\processed\features_used.txt
Anzahl Features: 28


## Fazit

- Rohdaten bleiben unverändert
- Feature Engineering ist bewusst moderat
- Ergebnis ist reproduzierbar und erklärbar

**Nächster Schritt:**
Notebook 01 – Modelltraining auf `diabetes_fe.csv`